# Python USB library for the Consumer Physics SCIO

__*IMPORTANT NOTE*__: This file contains the entire library (for testing and development purposes), but it is advisable to instead import the library. Refer to the other Jupyter notebook for that use.

### Dependencies
- pySerial

In [ ]:
import json
import struct
import logging
import serial
import serial.tools.list_ports as pyserial
#from serial.tools import list_ports_common # Not sure I need this?

# Logging/output format setup
log = logging.getLogger('root')
log.setLevel(logging.DEBUG)
logging.basicConfig(format='[%(asctime)s] %(levelname)8s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

In [76]:
# Search for device
def search_scio_usb(silent=False):
    if not silent: print('Searching for devices:')
    scio_ID = '0451:16AA'
    device_list = []
    # Find the scio port. Returns an empty list if not found
    scio_port = list(pyserial.grep(scio_ID))
    if(len(scio_port) == 0):
        print('  - Device not found. Try a long press (device should respond with blue blinking)')
        return(device_list)
    device_list.append({'address': scio_port[0].device, 'id': scio_ID})

    return(device_list)

class scio_usb:
    def __init__(self, address):
        self.address = address
        self.client = None
        self.device_info = {}
        self.disconnected = True
        self.cmd_base_protocol = -70 # 0xba
        self.cmd_dev_id =    1 # 0x01 # TODO: Unsure if this is correct
        self.cmd_scan   =    2 # 0x02
        self.cmd_temp   =    4 # 0x04
        self.cmd_bat    =    5 # 0x05
        self.cmd_led    =   11 # 0x0b
        self.cmd_ble_id = -124 # 0x84
        
    async def connect(self):
        try:
            self.client = serial.Serial(self.address)
            self.disconnected = False
        except Exception as e:
            logging.error(f'Failed to connect to {self.address}: {e}')
            raise
        pass
    
    async def disconnect(self):
        if not self.disconnected:
            try:
                self.client.close()
            except Exception as e:
                logging.error(f'Failed to disconnect from {self.address}: {e}')
                raise
            finally:
                self.disconnected = True
        pass
        
    def create_command(self, cmd):
        if(cmd == self.cmd_led):
            # Setting the LED colour is special (longer command)
            byte_cmd = struct.pack('<bbbbbbbbbbbbbbb',1,self.cmd_base_protocol,cmd,9,0,0,0,0,0,0,0,0,0,0)
        else:
            byte_cmd = struct.pack('<bbbbb',1,self.cmd_base_protocol,cmd,0,0)
        return(byte_cmd)
    
    async def send_command(self, byte_cmd):
        try:
            self.client.write(byte_cmd)
        except Exception as e:
            logging.error(f'Failed to send command to {self.address}: {e}')
            raise
        pass
    
    async def read_response(self):
        try:
            # Start reading the response
            s = self.client.read(1)
        except Exception as e:
            logging.error(f'Failed to read command from {self.address}: {e}')
            raise
        response_type = struct.unpack('<b',s)[0]
        # Error check
        assert (response_type == self.cmd_base_protocol), 'Wrong response type: {}'.format(response_type)
        # Read data
        s = self.client.read(1)
        response_content = struct.unpack('<b',s)[0]
        s = self.client.read(2)
        response_length = struct.unpack('<H',s)[0]
        # Read the number of values specified by the response length
        response_data = self.client.read(response_length)
        return([response_content, response_length, response_data])
    
    async def read_temperature(self):
        byte_cmd = self.create_command(self.cmd_temp)
        await self.send_command(byte_cmd)
        [response_content, response_length, response_data_raw] = await self.read_response()
        # Parse response
        num_vars = response_length / 4 # divide by 4 because we are dealing with longs
        data_struct = '<' + str(int(num_vars)) + 'L' # This is usually '<3l' or '<lll'
        # Convert bytes
        response_data = struct.unpack(data_struct, response_data_raw)
        # Convert to actual temperatures
        cmosTemperature   = (response_data[0] - 375.22) / 1.4092 # From the disassembled Android app
        chipTemperature   = response_data[1] / 100
        objectTemperature = response_data[2] / 100
        temperatures = {'cmos_t': cmosTemperature, 'chip_t': chipTemperature, 'obj_t': objectTemperature}
        return(temperatures)
    
    async def read_ble_id(self):
        byte_cmd = self.create_command(self.cmd_ble_id)
        await self.send_command(byte_cmd)
        [response_content, response_length, response_data_raw] = await self.read_response()
        # Parse response
        data_struct = '<10x40s16s64s'
        ble_data = list(struct.unpack(data_struct, response_data_raw))
        ble_data[0] = ble_data[0].decode('utf-8').strip()       # Serial number
        ble_data[1] = ble_data[1].decode('utf-8').strip('\x00') # Device name
        ble_data[2] = ble_data[2].decode('utf-8').strip()       # Production information
        # Store information
        self.device_info.update({'sn': ble_data[0], 'name': ble_data[1], 'i2s_tag_config': ble_data[2]})
        # TODO: Remaining, unknown bytes
        data_struct = '<10B'
        ble_data = list(struct.unpack(data_struct, response_data_raw[0:10])) + ble_data
        return(self.device_info)
    
    async def read_test(self):
        service_uuid = self.uuid_test
        current_service = self.base_service_uuid.replace('xxxx', service_uuid)
        
        try:
            test = await self.client.read_gatt_char(current_service)
            return test
        except Exception as e:
            logging.error(f'Failed to model nb from {self.address}: {e}')
            raise
        pass
    
    async def run(self, logfile, silent=True):
        if not silent: print('  - Connecting to device')
        await self.connect()
        
        ble_id = await self.read_ble_id()
        print(ble_id)
        
        raw_resp = await self.test_cmd(1)
        print(raw_resp)
        out = self.test_parse_response(raw_resp)
        print(out)
        
        temp = await self.read_temperature()
        if not silent: print('      CMOS T: {:.3f}°C'.format(temp['cmos_t'])) # cmosTemperature, chipTemperature, objectTemperature
        if not silent: print('      Chip T: {:.3f}°C'.format(temp['chip_t']))
        if not silent: print('      Obj. T: {:.3f}°C'.format(temp['obj_t']))
        # TEST
        #try:
        #    test = await self.read_test()
        #except:
        #    pass
        #print(len(test))
        #print(struct.unpack('<II', test))
        
        # Tell it to read temperature
        #await self.cmd_temperature()
        # Read the reply
        #test = await self.read_cmd_reply()
        #print(test)
        
        
        if not silent: print('  - Disconnecting')
        await self.disconnect()
        
        return
    
    async def test_cmd(self, cmd):
        byte_cmd = self.create_command(cmd)
        await self.send_command(byte_cmd)
        [response_content, response_length, response_data_raw] = await self.read_response()
        return([response_content, response_length, response_data_raw])
    
    # Generic, for testing
    def test_parse_response(self, response_data_raw):
        # Parse response
        data_struct = '<H'
        resp = struct.unpack(data_struct, response_data_raw[2][16:18])[0]
        #dev_id = struct.unpack(data_struct, response_data_raw)
        return(resp)

In [77]:
logfile = './logfile.csv'

# Search for devices
device_list = search_scio_usb()
if(len(device_list) != 0):
    # Note: No support for multiple Apogee devices. This takes the first one found!
    device_address = device_list[0]['address']
    print('  - Found device at', device_address)
    # Now set up device
    scio = scio_usb(device_address) # Create an instance for this device address
    await scio.run(logfile, silent=False)
print('Done...')

Searching for devices:
  - Found device at COM5
  - Connecting to device
{'sn': 'CPPCA0031C6PF0516009W6404386A1DF1816004A', 'name': 'myScio', 'i2s_tag_config': '20150812-e:PRODUCTION'}
[1, 28, b'\xe2M\xa2k#\x04\xc2\xc0\x00\x00\x8eL82\xe1\xc82\x80E\xab\x11a\xf1\x98\x93\x00\x01\x00']
32818
      CMOS T: 27.519°C
      Chip T: 33.500°C
      Obj. T: 0.000°C
  - Disconnecting
Done...


In [ ]:
device_list = await search_scio_ble(3)
print(device_list)